In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

In [75]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [76]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)

train_datagen = ImageDataGenerator() #no rescaling required as pre-trained model is already rescaling it
test_datagen = ImageDataGenerator()  #if we rescale again, we'll get bad accuracy


train_data = train_datagen.flow_from_directory('10_food_classes_all_data/train/',
                                   batch_size = 32,
                                  target_size=(128, 128),
                                   class_mode='categorical',
                                   shuffle = True, #shuffle is required because data is highly ordered into respective classes
                                   seed=42)
test_data = test_datagen.flow_from_directory('10_food_classes_all_data/test/',
                                  batch_size=32,
                                  target_size=(128, 128),
                                  class_mode='categorical',
                                  shuffle = True,
                                  seed=42)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [77]:
inputs = tf.keras.layers.Input(shape = (128, 128, 3))

#for Functional API, input layer and hidden layers should be separate otherwise hidden layers won't be able to take tensors

x = tf.keras.layers.Conv2D(64, kernel_size = 3)(inputs)
#like composition of functions, this style is called functional way of passing layers (Functional API)
x = tf.keras.layers.MaxPool2D(2)(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(10, activation = 'softmax')(x)

model = tf.keras.models.Model(inputs, outputs) #inputs and outputs functions from outside need to be connected (like a circle)

model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'] 
              )




In [78]:
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 63, 63, 64)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 254016)            0         
                                                                 
 dense_6 (Dense)             (None, 10)                2540170   
                                                                 
Total params: 2541962 (9.70 MB)
Trainable params: 2541962 (9.70 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [79]:
history = model.fit(train_data, epochs=5)

Epoch 1/5
235/235 [==============================] - 20s 85ms/step - loss: 596.3521 - accuracy: 0.1920
Epoch 2/5
235/235 [==============================] - 20s 83ms/step - loss: 78.3258 - accuracy: 0.4653
Epoch 3/5
235/235 [==============================] - 20s 86ms/step - loss: 39.7295 - accuracy: 0.6583
Epoch 4/5
235/235 [==============================] - 20s 84ms/step - loss: 18.6969 - accuracy: 0.7901
Epoch 5/5
235/235 [==============================] - 21s 87ms/step - loss: 15.8473 - accuracy: 0.8256


In [80]:
model.evaluate(test_data) #very bad accuracy

79/79 [==============================] - 6s 80ms/step - loss: 174.5893 - accuracy: 0.2224


[174.5893096923828, 0.2223999947309494]

In [81]:
base_model = tf.keras.applications.EfficientNetB0(include_top = False) #pre-trained model with hundreds of layers
#include_top = False doesn't import the input layer of the model as we are giving our model to it
base_model.trainable = False #we are not training base model with our inputs !!!

inputs = tf.keras.layers.Input(shape = (128, 128, 3))
x = base_model(inputs) #directly pass it through base model
x = tf.keras.layers.GlobalAveragePooling2D()(x)
#x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(10, activation = 'softmax')(x)
model = tf.keras.models.Model(inputs, outputs)

model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'] 
              )


In [82]:
model.summary() #we have more parameters here in pre-trained model compared to previous model

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d_4  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_7 (Dense)             (None, 10)                12810     
                                                                 
Total params: 4062381 (15.50 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 4049571 (15.45 MB)
_________________________________________________________________


In [83]:
model.fit(train_data, epochs = 5,
           validation_data = test_data,
           steps_per_epoch = len(train_data), #no. of steps taken on validation curve
           validation_steps = int(0.2 * len(test_data))) 
#only 20% of test data is chosen to learn => slowly and accurately learns it

#very bad accuracy


Epoch 1/5
235/235 [==============================] - 33s 128ms/step - loss: 1.0185 - accuracy: 0.6903 - val_loss: 0.5410 - val_accuracy: 0.8292
Epoch 2/5
235/235 [==============================] - 29s 123ms/step - loss: 0.6424 - accuracy: 0.7988 - val_loss: 0.5177 - val_accuracy: 0.8417
Epoch 3/5
235/235 [==============================] - 29s 125ms/step - loss: 0.5662 - accuracy: 0.8212 - val_loss: 0.4098 - val_accuracy: 0.8729
Epoch 4/5
 21/235 [=>............................] - ETA: 24s - loss: 0.5320 - accuracy: 0.8318

In [ ]:
model.evaluate(test_data) #good accuracy, all due to pre-trained model

79/79 [==============================] - 10s 130ms/step - loss: 0.4273 - accuracy: 0.8624


[0.4272502362728119, 0.8623999953269958]

In [ ]:
base_model.layers #shows base model layers

In [ ]:
for num, layer in enumerate(model.layers):
    print(num, layer.name, layer.trainable)

0 input_6 True
1 efficientnetb0 False
2 global_average_pooling2d_1 True
3 dense_3 True


In [ ]:
for num, layer in enumerate(base_model.layers):
    print(num, layer.name, layer.trainable)

0 input_5 False
1 rescaling_2 False
2 normalization_1 False
3 rescaling_3 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze Fals

In [ ]:
base_model.summary()
#here, rescaling is done by the model so we shouldn't rescale the dataset for this model 

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 rescaling_2 (Rescaling)     (None, None, None, 3)        0         ['input_5[0][0]']             
                                                                                                  
 normalization_1 (Normaliza  (None, None, None, 3)        7         ['rescaling_2[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_3 (Rescaling)     (None, None, None, 3)        0         ['normalization_1

In [ ]:
base_model_2 = tf.keras.applications.ResNet50(include_top = False) #another pre-trained model
base_model_2.trainable = False
base_model_2.summary()
#here, model doesn't rescale by itself => we need to rescale the dataset

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_8[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.random.set_seed(42)

train_datagen = ImageDataGenerator(rescale = 1./255) #no rescaling required as pre-trained model is already rescaling it
test_datagen = ImageDataGenerator(rescale = 1/255.)  #if we rescale again, we'll get bad accuracy


train_data = train_datagen.flow_from_directory('10_food_classes_all_data/train/',
                                   batch_size = 32,
                                  target_size=(128, 128),
                                   class_mode='categorical',
                                   shuffle = True, #shuffle is required because data is highly ordered into respective classes
                                   seed=42)
test_data = test_datagen.flow_from_directory('10_food_classes_all_data/test/',
                                  batch_size=32,
                                  target_size=(128, 128),
                                  class_mode='categorical',
                                  shuffle = True,
                                  seed=42)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
inputs = tf.keras.layers.Input(shape = (128, 128, 3))
x = base_model(inputs) #directly pass it through base model
x = tf.keras.layers.GlobalAveragePooling2D()(x)
#x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(10, activation = 'softmax')(x)
model = tf.keras.models.Model(inputs, outputs)

model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'] 
              )


In [ ]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_5 (Dense)             (None, 10)                12810     
                                                                 
Total params: 4062381 (15.50 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 4049571 (15.45 MB)
_________________________________________________________________


In [ ]:
model.fit(train_data, epochs = 5,
           validation_data = test_data,
           steps_per_epoch = len(train_data), #no. of steps taken on validation curve
           validation_steps = int(0.2 * len(test_data))) 
#only 20% of test data is chosen to learn => slowly and accurately learns it

#very bad accuracy, as the number of neurons in ResNet is lesser than that of EfficientNetB0


Epoch 1/5
235/235 [==============================] - 35s 136ms/step - loss: 2.3318 - accuracy: 0.1016 - val_loss: 2.3092 - val_accuracy: 0.1292
Epoch 2/5
235/235 [==============================] - 34s 143ms/step - loss: 2.3323 - accuracy: 0.0973 - val_loss: 2.3296 - val_accuracy: 0.0896
Epoch 3/5
235/235 [==============================] - 34s 143ms/step - loss: 2.3297 - accuracy: 0.0912 - val_loss: 2.3348 - val_accuracy: 0.0833
Epoch 4/5
235/235 [==============================] - 34s 145ms/step - loss: 2.3291 - accuracy: 0.1000 - val_loss: 2.3179 - val_accuracy: 0.1000
Epoch 5/5
235/235 [==============================] - 35s 146ms/step - loss: 2.3270 - accuracy: 0.1004 - val_loss: 2.3222 - val_accuracy: 0.0896


In [ ]:
len(base_model.layers) #number of layers

238

In [ ]:
len(base_model_2.layers) #number of layers

175

In [ ]:
base_model.trainable = True #every layer in base_model cannot be changeable
# => to make last 10 layers as changeable => need to unfreeze them with trainable = True 
for i, layer in enumerate(base_model.layers[:-10]):
    layer.trainable = False #last 10 layers are trainable(freezed)

In [ ]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, layer.trainable)

0 input_5 False
1 rescaling_2 False
2 normalization_1 False
3 rescaling_3 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze Fals

In [ ]:
#now last 10 layers of base model can have their weights changed
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 10)                12810     
                                                                 
Total params: 4062381 (15.50 MB)
Trainable params: 906042 (3.46 MB)
Non-trainable params: 3156339 (12.04 MB)
_________________________________________________________________


In [ ]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.0001), 
              #new lr = old lr / 10 because due to 10 freezed layers, model has become sensitive so we compensate for it
              metrics = ['accuracy'] 
              )

model.fit(train_data, epochs = 10,
           validation_data = test_data,
           steps_per_epoch = len(train_data), #no. of steps taken on validation curve
           validation_steps = int(0.2 * len(test_data)),
            initial_epoch = 5)

Epoch 6/10
117/235 [=============>................] - ETA: 15s - loss: 2.3035 - accuracy: 0.1155

KeyboardInterrupt: 

In [ ]:
'''
Steps in any Transfer Learning working:
1) Import the model
2) set trainable = False
3) Fit the model
4) Unfreeze last few rows
5) lr' = lr / 10 (lr - learning rate)
6) Fit the model
'''

In [1]:
import zipfile 
zip = zipfile.ZipFile("101_food_classes_10_percent.zip")
zip.extractall()
zip.close()

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.,
                                   rotation_range=0.6,
                                   shear_range = 0.7,
                                   zoom_range=0.5,
                                   horizontal_flip = True) #basic data augmentation

test_datagen = ImageDataGenerator(rescale=1/255.,
                                   rotation_range=0.6,
                                   shear_range = 0.7,
                                   zoom_range=0.5,
                                   horizontal_flip = True) #basic data augmentation

train_data = train_datagen.flow_from_directory('101_food_classes_10_percent/train',
                                               class_mode = 'categorical',
                                               batch_size = 32,
                                               shuffle = True,
                                               target_size = (128,128))

test_data = test_datagen.flow_from_directory('101_food_classes_10_percent/test',
                                               class_mode = 'categorical',
                                               batch_size = 32,
                                               shuffle = True,
                                               target_size = (128,128))

Found 7575 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.


In [5]:
base_model = tf.keras.applications.EfficientNetB0(include_top = False) #pre-trained model with hundreds of layers
#include_top = False doesn't import the input layer of the model as we are giving our model to it
base_model.trainable = False #we are not training base model with our inputs !!!

inputs = tf.keras.layers.Input(shape = (128, 128, 3))
x = base_model(inputs) #directly pass it through base model
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(101, activation = 'softmax')(x)
model = tf.keras.models.Model(inputs, outputs)

model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.legacy.Adam(),
              metrics = ['accuracy'] 
              )

hist = model.fit(train_data,
                 epochs = 5,
                 validation_data = test_data,
                 validation_steps = int(0.2 * len(test_data)))

#very very poor val_accuracy


Epoch 1/5
237/237 [==============================] - 73s 298ms/step - loss: 4.7222 - accuracy: 0.0075 - val_loss: 4.7193 - val_accuracy: 0.0087
Epoch 2/5
237/237 [==============================] - 71s 301ms/step - loss: 4.7106 - accuracy: 0.0087 - val_loss: 4.6875 - val_accuracy: 0.0097
Epoch 3/5
237/237 [==============================] - 69s 291ms/step - loss: 4.7141 - accuracy: 0.0104 - val_loss: 4.7006 - val_accuracy: 0.0093
Epoch 4/5
237/237 [==============================] - 67s 281ms/step - loss: 4.7142 - accuracy: 0.0088 - val_loss: 4.6903 - val_accuracy: 0.0093
Epoch 5/5
237/237 [==============================] - 66s 279ms/step - loss: 4.7155 - accuracy: 0.0096 - val_loss: 4.6934 - val_accuracy: 0.0113
